# 通过覆盖率的准则设计实例

In [ ]:
from typing import List
from typing import Optional
from typing import Dict

import os
import json
from pathlib import Path

import httpx
from openai import OpenAI

import requests

import time

## 进行谓词实例化

In [ ]:
# 文件路径
file_path = "prompts/DataFountain/predicates.txt"

# 请求的 URL
url = "https://api.moonshot.cn/v1/files"

headers = {
    'Authorization': 'sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw'
}

# 准备文件数据
files = {'file': open(file_path, 'rb')}

# 发起 POST 请求
response = requests.post(url, headers=headers, files=files)

# 打印响应内容
print(response.status_code)
print(response.text)

In [ ]:
client = OpenAI(
    api_key="sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw",
    base_url="https://api.moonshot.cn/v1",
)

file_object = client.files.create(file=Path("prompts/DataFountain/predicates.txt"), purpose="file-extract")
file_content = client.files.content(file_id=file_object.id).text

In [ ]:
print(file_content)

## 读取数据与格式转换

In [ ]:
import re

with open('data/DF/answers_782_v5.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [ ]:
print(data[0])

## JSON格式匹配

In [ ]:
def extract_json(text):
    # 使用正则表达式匹配并提取 JSON 数据（假设 JSON 数据用 `{` 开头和 `}` 结尾）
    json_data = re.search(r'\{.*\}', text, re.DOTALL)

    # 检查是否找到匹配的 JSON 数据
    if json_data:
        json_string = json_data.group(0)

        # # 将 JSON 数据解析为字典
        # data = json.loads(json_string)
        return json_string
    else:
        print("未找到 JSON 数据。")

In [ ]:
extract_json(data[0])

## 使用api进行谓词实例化

In [ ]:
def call_api_with_rate_limit_handling(client, messages, model="moonshot-v1-32k", max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            completion = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=0,
            )
            return completion
        except Exception as e:
            if '429' in str(e):  # 检查异常消息中是否包含429
                print("Rate limit reached. Retrying in 1 second...")
                time.sleep(1)  # 等待1秒钟
                retries += 1
            else:
                raise  # Re-raise the exception if it's not a 429 error
    raise Exception("Max retries reached. API call failed.")

In [ ]:
instance_predicates = []

In [ ]:
for index, text in enumerate(data, start=0):
    # 替换内容
    replacement = text

    # # 使用正则表达式替换 [[RULES]]
    # content = re.sub(r'\[\[RULES\]\]', replacement, file_content)

    # 使用replace
    content = file_content.replace("[[RULES]]", replacement)

    messages = [
        {
            "role": "system",
            "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。请将内容输出为可解析的json格式。"
        },
        {
            "role": "system",
            "content": content,
        },
        {
            "role": "user",
            "content": "你好，请根据txt内容生成回答。"
        }
    ]

    completion = call_api_with_rate_limit_handling(client, messages, "moonshot-v1-32k", 5)

    print(f"{index}")
    instance_predicates.append(completion.choices[0].message.content)

In [ ]:
with open('data/DF/instance_predicates_v2.json', 'w', encoding='utf-8') as f:
    json.dump(instance_predicates, f, ensure_ascii=False, indent=4)

In [ ]:
print(instance_predicates[1])

# 缓存转换

In [1]:
from typing import List
from typing import Optional
from typing import Dict

import os
import json
import jsonlines
from pathlib import Path

import httpx
from openai import OpenAI

import requests

import time
import re
from tqdm import tqdm

In [12]:
# 你的API密钥
api_key = "sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw"

# 读取文本文件内容
with open('prompts/DataFountain/predicates_v2.txt', 'r', encoding='utf-8') as file:
    file_content = file.read()

# 创建缓存的请求数据
data = {
    "model": "moonshot-v1",
    "messages": [
        {
            "role": "system",
            "content": file_content
        }
    ],
    "name": "example_cache",
    "ttl": 3600  # 缓存有效期，单位为秒
}

# 发送POST请求创建缓存
response = requests.post(
    url="https://api.moonshot.cn/v1/caching",
    headers={
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    },
    json=data
)

# 打印响应内容
print(json.loads(response.text))

{'id': 'cache-ezijgd5oc6di11gdfsh1', 'object': 'cache_object', 'model': 'moonshot-v1', 'messages': [{'role': 'system', 'content': '任务描述：你将被给予一条 JSON 格式的数据，包含一条规则（rule）的描述，这条规则中包含的谓词（predicates），以及根据规则转换而来的逻辑表达式（logicexpression）。你的任务是:\n1. 针对谓词取不同的具体的值，代入逻辑表达式中，其中取值要满足多样性与复杂性，不同的取值要尽可能的多；\n2. 对于谓词不同的取值请进行排列组合，保证尽可能出现所有取值的组合；\n3. 将实例化的逻辑表达式输出为可解析的 JSON 格式。\n------\n下面是一个例子：\n```json\n{\n    "rule": "安全生产协调司的职责：根据安全监管总局领导指示和有关规定，组织协调安全监察专员赶赴事故现场参与事故应急救援和事故调查处理工作。",\n    "predicates":\n    [\n        "指示：Instruction(Leader, Regulation) ::: 表示领导发出的指示和相关规定。",\n        "参与：Participate(Inspector, Scene, Task) ::: 表示安全监察专员在事故现场参与某项任务。",\n        "赶赴：GoTo(Inspector, Scene) ::: 表示安全监察专员赶赴事故现场。",\n        "事故：Accident(Scene) ::: 表示发生在某个场景的事故。"\n    ],\n    "logicexpression":\n    [\n        "∀s ∀i ∀r (Accident(s) ∧ Instruction(Leader, r) → (GoTo(i, s) ∧ Participate(i, s, EmergencyRescue) ∧ Participate(i, s, Investigation)))"\n    ]\n}\n```\n###\n这条例子的谓词实例化的参数值，和将实例化参数代入逻辑表达式的输出如下：\n[\n    {\n     

In [21]:
# 你的API密钥
api_key = "sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw"

# 缓存ID，假设从创建缓存的响应中获取
cache_id = "cache-ezj3acdoc6di11gdj58i"

# 发送DELETE请求删除缓存
response = requests.delete(
    url=f"https://api.moonshot.cn/v1/caching/{cache_id}",
    headers={
        "Authorization": f"Bearer {api_key}"
    }
)

# 打印响应状态码
print(response.status_code)

# 如果需要，也可以打印响应内容
print(response.text)

200
{"deleted":true,"id":"cache-ezj3acdoc6di11gdj58i","object":"cache_object.deleted"}


In [2]:
# 你的API密钥
api_key = "sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw"


# 读取文本文件内容
def load_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()


# 创建缓存
def create_cache(file_content):
    data = {
        "model": "moonshot-v1",
        "messages": [
            {
                "role": "system",
                "content": file_content
            }
        ],
        "name": "example_cache",
        "ttl": 3600  # 缓存有效期，单位为秒
    }
    response = requests.post(
        url="https://api.moonshot.cn/v1/caching",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    cache_response = json.loads(response.text)
    return cache_response['id']

In [3]:
# 检查缓存是否存在且未过期
def check_cache(cache_id):
    response = requests.get(
        url=f"https://api.moonshot.cn/v1/caching/{cache_id}",
        headers={
            "Authorization": f"Bearer {api_key}"
        }
    )
    if response.status_code == 200:
        return True
    return False


# 重新加载数据并更新缓存
def reload_and_update_cache(file_path, cache_id):
    new_file_content = load_file_content(file_path)
    data = {
        "model": "moonshot-v1",
        "messages": [
            {
                "role": "system",
                "content": new_file_content
            }
        ],
        "name": "example_cache",
        "ttl": 3600  # 缓存有效期，单位为秒
    }
    response = requests.put(
        url=f"https://api.moonshot.cn/v1/caching/{cache_id}",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    return response.status_code == 200


# 使用缓存内容并添加问题
def use_cache_with_question(cache_id, question):
    data = {
        "model": "moonshot-v1-32k",
        "messages": [
            {
                "role": "cache",
                "content": f"cache_id={cache_id};reset_ttl=3600",
            },
            {
                "role": "user",
                "content": question
            }
        ],
        "temperature": 0.7,
        "max_tokens": 8192,
    }
    response = requests.post(
        url="https://api.moonshot.cn/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    return response.json()['choices'][0]['message']['content']

## 主流程

In [15]:
def extract_json(text):
    # 使用正则表达式匹配并提取 JSON 数据（假设 JSON 数据用 `{` 开头和 `}` 结尾）
    json_data = re.search(r'\{.*\}', text, re.DOTALL)

    # 检查是否找到匹配的 JSON 数据
    if json_data:
        json_string = json_data.group(0)

        # # 将 JSON 数据解析为字典
        # data = json.loads(json_string)
        return json_string
    else:
        print("未找到 JSON 数据。")
        return None

In [4]:
# 主流程
file_path = 'prompts/DataFountain/predicates_v2.txt'
file_content = load_file_content(file_path)
print(f"File{file_path} success read")
cache_id = create_cache(file_content)
print(f"cache id:{cache_id}")
print("cache cuccess")

# # 模拟缓存过期
# time.sleep(3600)

# 检查缓存是否存在且未过期
if not check_cache(cache_id):
    print("检查不到缓存")
    # 重新加载数据并更新缓存
    if reload_and_update_cache(file_path, cache_id):
        print("缓存更新成功")
    else:
        print("缓存更新失败")

print("缓存已存在")

with open('data/RJUA/RJUA_nl2logic_v5.json', 'r', encoding='utf-8') as file:
    answer = json.load(file)

for idx, value in tqdm(enumerate(answer[:100]), total=len(answer), desc="Processing"):
    context = value['input']
    # 使用缓存内容并添加问题
    question = f"请回答关于文件内容的问题，其中[[CONTEXT]]代表的数据为{context}"
    response = use_cache_with_question(cache_id, question)
    judgement = {
        "ori_id": value["ori_id"],
        "number": value['number'],
        "rule": context["rule"],
        "predicates": response
    }
    # 打开文件以进行写入，如果文件不存在，会创建文件
    with jsonlines.open('data/RJUA/RJUA_logic2predicates_100.jsonl', mode='a') as writer:
        writer.write(judgement)


Fileprompts/DataFountain/predicates_v2.txt success read
cache id:cache-ezth445oc6di11ggieci
cache cuccess
缓存已存在


Processing:  12%|█▏        | 100/838 [1:59:34<14:42:29, 71.75s/it]
